In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, DataCollatorWithPadding, BertForSequenceClassification, get_scheduler
import wandb
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate
import torch
from tabulate import tabulate
from torch.optim import SGD
from accelerate import Accelerator

class Accumulator:
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def tokenize_function(example):
    result = tokenizer(example["text"], truncation=True)
    result["labels"] = example["checkworthiness"]
    return result

dataset = load_dataset("Nithiwat/claim-detection")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text", "checkworthiness"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/873 [00:00<?, ?B/s]

data/train-00000-of-00001-b257f2fa7963ab(…):   0%|          | 0.00/1.59M [00:00<?, ?B/s]

data/test-00000-of-00001-d48b0506d97e25d(…):   0%|          | 0.00/492k [00:00<?, ?B/s]

data/valid-00000-of-00001-aaf59ec2907a08(…):   0%|          | 0.00/396k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/23276 [00:00<?, ? examples/s]

Map:   0%|          | 0/7274 [00:00<?, ? examples/s]

Map:   0%|          | 0/5819 [00:00<?, ? examples/s]

In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(11200))
small_eval_dataset = tokenized_datasets["test"]

train_dataloader = DataLoader(
    small_train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    small_eval_dataset, batch_size=16, collate_fn=data_collator
)

In [4]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader) # 1400
num_adam_steps = num_training_steps // 5 * 4 # 1120, 280

run = wandb.init(project="bert-fine-tuning", name="bert-loop-analysis")
progress_bar = tqdm(range(num_training_steps))
table_data = [["Step", "Training Loss", "Validation Loss", "Accuracy", "F1"]]
step_count = 0

adamW = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
sgd = SGD(model.parameters(), lr=2e-5, weight_decay=1e-2)
adamW_scheduler = get_scheduler("cosine", optimizer=adamW, num_warmup_steps=300, num_training_steps=num_adam_steps)
sgd_scheduler = get_scheduler("cosine", optimizer=sgd, num_warmup_steps=50, num_training_steps=num_training_steps - num_adam_steps)

accelerator = Accelerator()
model, adamW, sgd, adamW_scheduler, sgd_scheduler, train_dataloader, eval_dataloader = accelerator.prepare(
    model, adamW, sgd, adamW_scheduler, sgd_scheduler, train_dataloader, eval_dataloader
)

# 共 11200 个样本，每次 16 个样本进行训练，共 700 个批次，训练轮数 2， 共 1400 个批次，每 50 个 批次进行一次评估，共评估 1400/50=28 次
for epoch in range(num_epochs): # 共 1200 steps, 每轮 600 个批次,
    model.train()

    train_metric = Accumulator(2)
    for i, train_batch in enumerate(train_dataloader):
        step_count += 1
        outputs = model(**train_batch)
        train_loss = outputs.loss
        train_metric.add(train_loss.item(), 1)

        accelerator.backward(train_loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        if step_count > num_adam_steps:
            sgd.step()
            sgd_scheduler.step()
            sgd.zero_grad()
        else:
            adamW.step()
            adamW_scheduler.step()
            adamW.zero_grad()

        progress_bar.update(1)

        # evaluation
        if (i + 1) % 50 == 0 or i == len(train_dataloader) - 1:
            train_loss_avg = train_metric[0] / train_metric[1]

            accuracy_metric = evaluate.load("accuracy")
            f1_metric = evaluate.load("f1")

            model.eval()
            eval_metric = Accumulator(2)
            for j, eval_batch in enumerate(eval_dataloader):
                with torch.no_grad():
                    outputs = model(**eval_batch)

                eval_loss = outputs.loss
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)

                eval_metric.add(eval_loss.item(), 1)
                accuracy_metric.add_batch(predictions=predictions.cpu().numpy(), references=eval_batch["labels"].cpu().numpy())
                f1_metric.add_batch(predictions=predictions.cpu().numpy(), references=eval_batch["labels"].cpu().numpy())

            eval_loss_avg = eval_metric[0] / eval_metric[1]
            accuracy = accuracy_metric.compute()
            f1 = f1_metric.compute()

            print(f"\nsteps: {step_count}, eval_loss_avg:{eval_loss_avg}, train loss:{train_loss_avg}, acc: {accuracy['accuracy']}, f1: {f1['f1']}")

            table_data.append([
                step_count,
                f"{train_loss_avg}",
                f"{eval_loss_avg}",
                f"{accuracy['accuracy']}",
                f"{f1['f1']}"
            ])

            wandb.log({"train/loss": train_loss_avg,
                       "eval/loss": eval_loss_avg,
                       "eval/accuracy": accuracy,
                       "eval/f1": f1,
                       })
            train_metric.reset()

print(tabulate(table_data, headers="firstrow", tablefmt="grid"))
run.finish()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xiaojingellenliu (xiaojingellenliu-ellen) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/1400 [00:00<?, ?it/s]


steps: 50, eval_loss_avg:0.6326457974674937, train loss:0.6809298598766327, acc: 0.5130602144624691, f1: 0.6757004211682842

steps: 100, eval_loss_avg:0.3971071686063494, train loss:0.5184678238630295, acc: 0.867748144074787, f1: 0.8705248990578734

steps: 150, eval_loss_avg:0.30089638432958626, train loss:0.36307030022144315, acc: 0.8812207863623865, f1: 0.875968992248062

steps: 200, eval_loss_avg:0.3276375254841296, train loss:0.2948821370303631, acc: 0.8667858124828155, f1: 0.8526235741444866

steps: 250, eval_loss_avg:0.2546183136383911, train loss:0.2729119826853275, acc: 0.8948309045916965, f1: 0.8920863309352518

steps: 300, eval_loss_avg:0.25281525899420726, train loss:0.2760804694890976, acc: 0.896618091833929, f1: 0.8982959156072491

steps: 350, eval_loss_avg:0.23417576627990047, train loss:0.2543842035531998, acc: 0.9059664558702227, f1: 0.9054465026264861

steps: 400, eval_loss_avg:0.23453891223276055, train loss:0.2510954497009516, acc: 0.9019796535606269, f1: 0.90514833

eval/loss,█▄▂▃▂▂▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁
train/loss,█▆▄▃▃▃▃▃▃▃▃▃▂▃▂▁▁▁▁▁▁▁▂▁▁▁▁▁
eval/loss,0.23642
train/loss,0.12277
